In [ ]:
import os
import csv
import pandas as pd
import numpy as np
from numpy import fft
from scipy.fftpack import fft
from scipy.fft import fft, fftfreq
# import scipy.integrate as spi
import matplotlib
from  matplotlib import pyplot as plt
# import librosa
from ipywidgets import interact, fixed, FloatSlider
import IPython.display as ipd
from sklearn import preprocessing
from BaselineRemoval import BaselineRemoval
from scipy import sparse
from scipy.sparse.linalg import spsolve
import datetime
import time
# import mysql.connector
# from mysql.connector import Error
%matplotlib inline




In [ ]:
def connect_db():
    # 連接資料庫
    import mysql.connector
    from mysql.connector import Error

    try:
        connection = mysql.connector.connect(
            host='35.77.173.217', #資料庫的host
#             port='3306', #資料庫的port (通常會是3306)
            database='Laundry', #資料庫名稱# database='Laundry', #資料庫名稱
            user='laundry', #資料庫使用者
            password='12345678',# password='12345678', #資料庫密碼
        )
        if connection.is_connected():
            cursor = connection.cursor()
            print('connected to ',connection.get_server_info())

    except Error as e:
        print('資料庫連接失敗,'+'\n'+'錯誤代碼為 '+e)

    return connection, cursor    

CONNECTION, CURSOR = connect_db()

In [ ]:
with open('0819_kc_nano_result_1.csv','rt') as csvfile:
    rows = csv.reader(csvfile)
    df=pd.read_csv(r'./0819_kc_nano_result_1.csv')
    data_arr = []
    data_arrx = []
    data_arry = []
    data_arrxt = []
    for row in rows:
        data_arr.append(row)
    for i in range(1,len(df)+1):
        data_arrx.append(float(data_arr[i][0])-float(data_arr[1][0]))
#         data_arrx.append(float(data_arr[i][0]))
        data_arry.append(float(data_arr[i][3]))
#         data_arrxt.append(float(data_arr[i][0])-float(data_arr[1][0]))
#     data_arry = preprocessing.scale(data_arry)
#     for i in range(1, len(df)-1):
#         data_arrx[i] = (data_arrx[i+1]-data_arrx[i])
#         print(data_arrx[i])
#         if i == 1:
#             data_arry[1] = data_arry[1]*data_arrx[1]
#             print (data_arry[i])
#         else :
#             data_arry[i] = data_arry[i-1] + data_arry[i]*data_arrx[i]
#             print (data_arry[i])
    print(len(data_arry))

In [ ]:
fig = plt.figure()
ax = plt.axes()
x = np.array(data_arrx)
y = np.array(data_arry)
# print (y[0:5])
# print (np.percentile(y[0:5], 50))
# print (sum(y[0:5])/5)
# print (np.mean(y[0:5]))
# print (np.std(y[0:5]))
# y = preprocessing.scale(y)
print(len(x))
print(len(y))
# print (np.percentile(y[0:5], 50))
plt.xticks(np.arange(0, len(x)+1, 500))
ax.plot(x,y,color = 'k', linewidth=0.1)


In [ ]:
CONNECTION, CURSOR = connect_db()
import time
from scipy import sparse
from scipy.sparse.linalg import spsolve
#128 datas  = 23.68sec
data_128 = np.array([])
copy = np.array([])
freq = []
T = 0.038
N = len(x)
zeros = np.zeros(2000)
half_c =(np.hanning(208))
mean = np.mean(data_arry[100:250])
temp = np.array([])
def get_current_time():
    print ('li')
    current_time = str(datetime.datetime.now()).split('.')[0]
    return current_time

def countdown(num_of_secs):
    i = 0
    
    while num_of_secs:
        global current_time
        m, s = divmod(num_of_secs, 60)
        min_sec_format = '{:02d}:{:02d}'.format(m, s)
#         print(min_sec_format, end='/r')
        print(min_sec_format)
        if i % 5 == 0:
            
            print ('hi')
            print ('洗衣中')
            current_time = get_current_time()
            print (current_time)
            sql = "INSERT INTO `washer_1` (`state`, `left_time`, `time`) VALUES ('洗衣中','"+str(min_sec_format)+"', '"+str(current_time)+"'); " 
            CURSOR.execute(sql) 
            CONNECTION.commit()
        i += 1
        time.sleep(1)
        num_of_secs -= 1
        
    print('00:00')
    print ('空機')
    current_time = get_current_time()
    sql = "INSERT INTO `washer_1` (`state`, `left_time`, `time`) VALUES ('空機中','00:00', '"+str(current_time)+"'); " 
    CURSOR.execute(sql) 
    CONNECTION.commit()

# print (y[0:1000])
# y2 = preprocessing.scale(y)
# print (y2[0:1000])
# y = preprocessing.scale(y)
print (mean)

# def baseline_als(y, lam, p, niter=10):
#     L = len(y)
#     D = sparse.csc_matrix(np.diff(np.eye(L), 2))
#     w = np.ones(L)
#     for i in range(niter):
#         W = sparse.spdiags(w, 0, L, L)
#         Z = W + lam * D.dot(D.transpose())
#         z = spsolve(Z, w*y)
#         w = p * (y > z) + (1-p) * (y < z)
#     return z

# plt.plot(half_c)
for i in range(0,len(data_arry)):
    
    if data_128.size<208:
        data_128 = np.append(data_128,y[i])
        copy = np.append(copy,y[i])
    else:
        data_128 = np.delete(data_128,0)
        data_128 = np.append(data_128,y[i])
         
        copy = np.delete(copy,0)
        copy = np.append(copy,y[i]) 
    if len(data_128)==208 and i!=0 and i%26==0:
#         plt.figure(figsize=(8, 2))
        data_128 = np.copy(copy)
#         offset = baseline_als(data_128, 10000, 0.001, niter=10)
        
#         print (offset)
#         offset = np.array(offset)
#         mean = np.mean(offset)
#         data_128 = np.subtract(data_128, mean)
#         print (data_128)
#         data_128 = np.add(data_128, -mean)
#         data_128 = preprocessing.scale(data_128)
        
        polynomial_degree=2 #only needed for Modpoly and IModPoly algorithm
        baseObj=BaselineRemoval(data_128)        
        data_128=baseObj.IModPoly(polynomial_degree)
#         data_128 = np.multiply(data_128,5)
#         data_128 = np.add(data_128, -0.095)
        
        
#         plt.figure(figsize=(8, 2))
#         plt.xticks(np.arange(0, len(data_128)+1, 500))
# #         x1 = len(data_128)
#         x1 = np.linspace(0,len(data_128),len(data_128))
#         y1 = data_128
#         plt.plot(x1,y1,color = 'k', linewidth=0.1)

        

        data_128 = np.multiply(data_128,half_c)
        data_128 = np.append(data_128,zeros)
#         N = 181
#         x = np.linspace(0.0, N*T, N, endpoint=False)
#         y = np.sin(50.0 * 2.0*np.pi*x) + 0.5*np.sin(80.0 * 2.0*np.pi*x)
#         yf = fft(data_128)
#         xf = fftfreq(data_128.size, T)[:N//2]
        yf = 1/data_128.size * np.abs(fft(data_128)[0:data_128.size//2])
        xf =(fftfreq(data_128.size,T)[:data_128.size//2])
#         for i in range(1,len(xf)):
#             if xf[i] == 0.5:
#                 print (i)
        yf[0:50] = 0
        index = np.where(yf == max(yf[:]))
        
#         print (xf)
        
#         n = 250
#         idx = np.argpartition(yf, -n)[-n:]
#         indices = idx[np.argsort((-yf)[idx])]
#         frequency = 0
#         for z in range(0,250):
#             frequency += xf[indices[z]]
#         frequency/250
#         print(index)
#         print(xf[index[0][0]])
#         print(yf[index[0][0]])
        data_128 = np.delete(data_128,np.s_[208:2000+208])
        freq.append(xf[index[0]])
#         plt.figure(figsize=(8, 2))
#         plt.plot(xf,yf)
# #         plt.plot(xf, 2.0/data_128.size * np.abs(yf[0:data_128.size//2]))
# #         plt.plot(data_128)
#         plt.grid()
#         plt.tight_layout()

# print (freq[0])

plt.figure(figsize = (20,5))
X = np.linspace(0,N*T,len(freq))
Y = np.array(freq)
# plt.xticks(np.arange(0, 2000, 50))
plt.yticks(np.arange(0, 20, 0.5))
plt.grid(linewidth= 0.2)
plt.plot(X,Y,c='k',linewidth=0.2)
plt.xlabel('Time()millisecond (s)')
plt.ylabel('Frequency (Hz)')
# plt.title("0811_result_2_z : window 750 ")
plt.title("0819_result_1 : window 175")
print (len(freq))

data_10 = np.array([])
copy_10 = np.array([])
freq_10 = []
median = 0
count = 0


for i in range(0,len(freq)):
    
    if data_10.size < 110:
        data_10 = np.append(data_10,freq[i])
    else:
        data_10 = np.delete(data_10,0)
        data_10 = np.append(data_10,freq[i])
         
    if len(data_10)==110:
#         print (data_10)
#         print (np.percentile(data_10, 50))
        freq_10 = np.append(freq_10, np.percentile(data_10, 50)) #中位數
#         freq_10 = np.append(freq_10, np.mean(data_10))
#         print (freq_10)

        if 6 < np.percentile(data_10, 50) < 8:
            count += 1
            print (count)
            print ('*')
            
            if count > 200:
                print ("注水中")
                print ("總時間=", count/0.102)
                print ("剩餘時間=", (count/0.102)-count)

        
        else:
            count = 0
            print ('count')
            print (count)
            
        if len(temp)<2 :
            temp = np.append(temp,count)
        elif len(temp)==2 :
            temp = np.delete(temp,0)
            temp = np.append(temp,count)
            
        if temp[0] > 200 and temp[1] == 0:
            countdown(int((temp[0]/0.102)-temp[0]))
    
        
plt.figure(figsize = (20,5))
X = np.linspace(0,N*T/10,len(freq_10))
Y = np.array(freq_10)
plt.xticks(np.arange(0, 2000, 50))
plt.yticks(np.arange(0, 20, 0.5))
plt.grid(linewidth= 0.2)
plt.plot(X,Y,c='k',linewidth=0.2)
plt.xlabel('Time()milliseconds (10s)')
plt.ylabel('Frequency (Hz)')
# plt.title("0811_result_2_z : window 750-30 ")
plt.title("0819_result_1  : window 175-100")
print (len(freq_10))
print (N*T/10)